In [210]:
import pandas as pd
import funciones.test_func as tf
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv()

host=os.getenv("DB_HOST2")
port=os.getenv("DB_PORT2")
database=os.getenv("DB_NAME2")
user=os.getenv("DB_USER2")
password=os.getenv("DB_PASS2")

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

tables = tf.excecute_query_tables(engine)

dfs = tf.create_df_variable(tables, engine)
patients = dfs['patients']
doctors= dfs['doctors']
appointments = dfs['appointments']
treatments = dfs['treatments']

In [211]:
patients

,patient_id,name,gender,birth_date
0,1,Ana Torres,F,1985-07-10
1,2,Luis Martínez,M,1970-02-20
2,3,Carlos Gómez,M,1995-11-15
3,4,Marta Ruiz,F,2001-01-05
4,5,Julia Herrera,F,1968-04-28
5,6,Pedro López,M,1988-06-12
6,7,Sandra Díaz,F,1992-09-30
7,8,Andrés Pérez,M,1975-12-19
8,9,Laura Castro,F,2000-03-21
9,10,José Molina,M,1963-08-08


In [212]:
doctors

,doctor_id,name,specialty
0,1,Dr. Pérez,Cardiology
1,2,Dr. Rojas,Neurology
2,3,Dr. Torres,General
3,4,Dr. Álvarez,Oncology
4,5,Dr. Salinas,Dermatology
5,6,Dr. Fernández,Pediatrics


In [213]:
appointments

,appointment_id,patient_id,doctor_id,appointment_date,status
0,101,1,1,2023-05-10,Completed
1,102,2,2,2023-05-15,Completed
2,103,3,3,2023-06-01,Cancelled
3,104,1,1,2023-06-20,Completed
4,105,4,2,2023-06-22,Completed
5,106,5,4,2023-07-05,Completed
6,107,5,4,2023-07-20,Completed
7,108,2,1,2023-08-10,Completed
8,109,6,5,2023-09-01,Cancelled
9,110,7,3,2023-09-10,Completed


In [214]:
treatments

,treatment_id,patient_id,illness,start_date,end_date,outcome
0,201,1,Hypertension,2023-05-10,2023-07-01,Recovered
1,202,2,Stroke,2023-05-15,2023-08-15,Recovered
2,203,5,Cancer,2023-07-05,2023-10-10,Deceased
3,204,4,Migraine,2023-06-22,2023-07-22,Recovered
4,205,6,Eczema,2023-09-01,2023-10-01,Recovered
5,206,7,Diabetes,2023-09-10,2023-11-15,Ongoing
6,207,8,Leukemia,2023-09-12,2023-12-20,Ongoing
7,208,9,Acne,2023-10-01,2023-11-01,Recovered
8,209,10,Arrhythmia,2023-10-15,2023-11-15,Recovered
9,210,11,Asthma,2023-10-20,2023-12-01,Recovered


Challenge #4: Challenge: Treatment Effectiveness & Patient Journey
🩺 Scenario:
The hospital wants to analyze the effectiveness of treatments, the efficiency of doctors, and patterns in appointments to improve patient care.

You are tasked with analyzing the performance based on the data available in the following tables:

- patients
- doctors
- appointments
- treatments

🔧 1. Doctor Effectiveness Analysis
Goal: Find which doctor has treated the most patients that fully recovered and which one has the highest recovery rate (recovered / total patients treated).

Questions:

a. Which doctor treated the most recovered patients?

In [215]:
#SQL

query = """
WITH recovered_patients AS (
SELECT patient_id 
FROM treatments
WHERE outcome = 'Recovered'
)

SELECT d.name, COUNT(DISTINCT a.patient_id) AS recovered_patients
FROM appointments a
JOIN recovered_patients rp ON rp.patient_id = a.patient_id
JOIN doctors d ON d.doctor_id = a.doctor_id
GROUP BY d.name
ORDER BY COUNT(DISTINCT a.patient_id) DESC
LIMIT 1
"""

tf.excecute_query(query, engine)

,name,recovered_patients
0,Dr. Pérez,3


In [216]:
#Python

df = pd.merge(pd.merge(treatments[['outcome', 'patient_id']], appointments[['patient_id', 'doctor_id']], on='patient_id'), doctors[['name', 'doctor_id']], on='doctor_id')
df_recovered_patients = df[df['outcome']=='Recovered']
df_recovered_patients = df_recovered_patients.groupby(['doctor_id','name'])['patient_id'].nunique().reset_index().rename(columns={'patient_id':'recovered_patients'})
df_recovered_patients = df_recovered_patients.sort_values(by='recovered_patients', ascending=False).drop(columns='doctor_id')
df_recovered_patients.head(1)

,name,recovered_patients
0,Dr. Pérez,3


b. Which doctor has the highest recovery rate?

In [217]:
#SQL

query = """
WITH recovered_patients AS (
SELECT patient_id 
FROM treatments
WHERE outcome = 'Recovered'
),

recovered_from_each_doctor AS (
SELECT d.doctor_id, d.name, COUNT(DISTINCT a.patient_id) AS recovered_patients
FROM appointments a
JOIN recovered_patients rp ON rp.patient_id = a.patient_id
JOIN doctors d ON d.doctor_id = a.doctor_id
GROUP BY d.doctor_id, d.name
ORDER BY COUNT(DISTINCT a.patient_id) DESC
),

treated_from_each_doctor AS (
SELECT d.doctor_id, d.name, COUNT(DISTINCT a.patient_id) AS treated_patients
FROM appointments a
JOIN treatments t ON t.patient_id = a.patient_id
JOIN doctors d ON d.doctor_id = a.doctor_id
GROUP BY d.doctor_id, d.name
ORDER BY COUNT(DISTINCT a.patient_id) DESC
)

SELECT tfed.name, recovered_patients, treated_patients, ROUND(1.0 * recovered_patients/treated_patients,2) AS recovery_rate
FROM treated_from_each_doctor tfed
JOIN recovered_from_each_doctor rfed ON tfed.doctor_id = rfed.doctor_id
ORDER BY recovery_rate DESC
LIMIT 1
"""

tf.excecute_query(query, engine)

,name,recovered_patients,treated_patients,recovery_rate
0,Dr. Salinas,2,2,1.0


In [218]:
#Python

df = pd.merge(pd.merge(treatments[['outcome', 'patient_id']], appointments[['patient_id', 'doctor_id']], on='patient_id'), doctors[['name', 'doctor_id']], on='doctor_id')
df_recovered_patients = df[df['outcome']=='Recovered']
df_recovered_patients = df_recovered_patients.groupby(['doctor_id','name'])['patient_id'].nunique().reset_index().rename(columns={'patient_id':'recovered_patients'})
df_treated_patients = df.groupby('doctor_id')['patient_id'].count().reset_index().rename(columns={'patient_id':'treated_patients'})
df_recovery_rate = pd.merge(df_recovered_patients, df_treated_patients, on= 'doctor_id')
df_recovery_rate['recovery_rate'] = df_recovery_rate['recovered_patients'] / df_recovery_rate['treated_patients'] 
df_recovery_rate['recovery_rate'] = df_recovery_rate['recovery_rate'].round(2)
df_recovery_rate = df_recovery_rate.sort_values(by='recovery_rate', ascending=False).drop(columns='doctor_id')
df_recovery_rate.head(1)

,name,recovered_patients,treated_patients,recovery_rate
2,Dr. Salinas,2,3,0.67


2. Missed Appointments Detection
Goal: Understand which patients have cancelled appointments and how often.

Questions:

a. List all patients who have cancelled more than 1 appointment.

In [219]:
#SQL

query = """
SELECT p.patient_id, p.name, COUNT(status) AS cancelled_appointments
FROM patients p 
JOIN appointments a ON a.patient_id = p.patient_id
WHERE status = 'Cancelled' 
GROUP BY p.patient_id, p.name
HAVING COUNT(status) > 1
ORDER BY COUNT(status) DESC
"""

tf.excecute_query(query, engine)

,patient_id,name,cancelled_appointments
0,5,Julia Herrera,3
1,2,Luis Martínez,2
2,3,Carlos Gómez,2
3,6,Pedro López,2


In [220]:
#Python

df_cancelled = pd.merge(patients[['patient_id', 'name']], appointments[['patient_id', 'status']], on='patient_id') 
df_cancelled = df_cancelled[df_cancelled['status'] == 'Cancelled']
df_cancelled = df_cancelled.groupby(['patient_id','name'])['status'].count().reset_index().sort_values(by='status', ascending=False)
df_cancelled = df_cancelled.rename(columns={'status':'cancelled_appointments'})
df_cancelled[df_cancelled['cancelled_appointments'] >1]

,patient_id,name,cancelled_appointments
4,5,Julia Herrera,3
1,2,Luis Martínez,2
2,3,Carlos Gómez,2
5,6,Pedro López,2


b. Which doctor has the highest number of cancelled appointments?

In [221]:
#SQL

query = """
SELECT d.doctor_id, d.name, COUNT(status) AS cancelled_appointments
FROM doctors d 
JOIN appointments a ON a.doctor_id = d.doctor_id
WHERE status = 'Cancelled' 
GROUP BY d.doctor_id, d.name
ORDER BY COUNT(status) DESC
LIMIT 1
"""

tf.excecute_query(query, engine)

,doctor_id,name,cancelled_appointments
0,4,Dr. Álvarez,4


In [222]:
#Python

df_cancelled = pd.merge(doctors[['doctor_id', 'name']], appointments[['doctor_id', 'status']], on='doctor_id') 
df_cancelled = df_cancelled[df_cancelled['status'] == 'Cancelled']
df_cancelled = df_cancelled.groupby(['doctor_id','name'])['status'].count().reset_index().sort_values(by='status', ascending=False)
df_cancelled = df_cancelled.rename(columns={'status':'cancelled_appointments'})
df_cancelled[df_cancelled['cancelled_appointments'] > 1].head(1)

,doctor_id,name,cancelled_appointments
3,4,Dr. Álvarez,4


3. Treatment Duration Insight
Goal: Calculate how long (in days) each patient spent in treatment.

Questions:

a. Add a column to calculate the duration of treatment.

b. What is the average treatment duration for each illness?

In [223]:
#SQL

query = """
SELECT t.illness, AVG((CAST(end_date AS date) - CAST(start_date AS date))) AS avg_treatment_duration
FROM treatments t
GROUP BY t.illness
ORDER BY avg_treatment_duration DESC
"""

tf.excecute_query(query, engine)

,illness,avg_treatment_duration
0,Leukemia,99.0
1,Cancer,97.0
2,Stroke,92.0
3,Diabetes,66.0
4,Melanoma,61.0
5,Hypertension,52.0
6,Asthma,42.0
7,Epilepsy,40.0
8,Arrhythmia,31.0
9,Acne,31.0


In [ ]:
#Python

treatments_avg_duration = treatments.copy()
treatments_avg_duration[['start_date', 'end_date']] = treatments_avg_duration[['start_date', 'end_date']].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors='coerce'))
treatments_avg_duration['avg_treatment_duration'] = treatments_avg_duration['end_date'] - treatments_avg_duration['start_date']
treatments_avg_duration.groupby('illness')['avg_treatment_duration'].mean().reset_index().sort_values(by='avg_treatment_duration', ascending=False)

,illness,avg_treatment_duration
8,Leukemia,99 days
3,Cancer,97 days
11,Stroke,92 days
4,Diabetes,66 days
9,Melanoma,61 days
7,Hypertension,52 days
2,Asthma,42 days
6,Epilepsy,40 days
0,Acne,31 days
1,Arrhythmia,31 days


c. What is the longest treatment duration and for which illness and patient?

In [225]:
#SQL

query = """
SELECT p.patient_id, p.name, t.illness, (CAST(end_date AS date) - CAST(start_date AS date)) AS treatment_duration
FROM treatments t
JOIN patients p on p.patient_id = t.patient_id
ORDER BY treatment_duration DESC
LIMIT 1
"""

tf.excecute_query(query, engine)

,patient_id,name,illness,treatment_duration
0,8,Andrés Pérez,Leukemia,99


In [ ]:
#Python

treatments_duration = treatments.copy()
treatments_duration[['start_date', 'end_date']] = treatments_duration[['start_date', 'end_date']].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors='coerce'))
treatments_duration['treatment_duration'] = treatments_duration['end_date'] - treatments_duration['start_date']
treatments_duration = treatments_duration.sort_values(by='treatment_duration', ascending=False)
treatments_duration = pd.merge(treatments_duration, patients[['patient_id', 'name']], on='patient_id')
treatments_duration[['patient_id', 'name', 'illness', 'treatment_duration']].head(1)

,patient_id,name,illness,treatment_duration
0,8,Andrés Pérez,Leukemia,99 days


4. Patient Journey Timeline
Goal: Build a timeline view of a patient's interactions with the hospital.

Questions:

a. For each patient, what is the time gap between their first appointment and treatment start?

In [251]:
#SQL

query = """
WITH first_appointment_patient AS (
SELECT patient_id, MIN(CAST(appointment_date AS date)) AS first_appointment
FROM appointments
GROUP BY patient_id
)

SELECT p.name, t.patient_id, t.start_date, fa.first_appointment, (CAST(t.start_date AS date) - fa.first_appointment) AS timegap
FROM treatments t
JOIN first_appointment_patient fa ON fa.patient_id = t.patient_id
JOIN patients p ON p.patient_id = t.patient_id
"""

tf.excecute_query(query, engine)

,name,patient_id,start_date,first_appointment,timegap
0,Ana Torres,1,2023-05-10,2023-05-10,0
1,Luis Martínez,2,2023-05-15,2023-05-15,0
2,Marta Ruiz,4,2023-06-22,2023-06-22,0
3,Julia Herrera,5,2023-07-05,2023-07-05,0
4,Pedro López,6,2023-09-01,2023-09-01,0
5,Sandra Díaz,7,2023-09-10,2023-09-10,0
6,Andrés Pérez,8,2023-09-12,2023-09-12,0
7,Laura Castro,9,2023-10-01,2023-10-01,0
8,José Molina,10,2023-10-15,2023-10-15,0
9,Lucía Peña,11,2023-10-20,2023-10-20,0


In [ ]:
#Python

treatments_by_patient = treatments.copy()
treatments_by_patient['start_date'] = pd.to_datetime(treatments_by_patient['start_date'], format='%Y-%m-%d', errors='coerce')
first_appointment_patient = appointments.copy()
first_appointment_patient['appointment_date'] = pd.to_datetime(first_appointment_patient['appointment_date'], format='%Y-%m-%d', errors='coerce')
first_appointment_patient = first_appointment_patient.groupby('patient_id')['appointment_date'].min().reset_index()
first_appointment_patient = first_appointment_patient.rename(columns={'appointment_date':'first_appointment'})
df_merged = pd.merge(pd.merge(treatments_by_patient[['patient_id', 'start_date']], first_appointment_patient[['patient_id', 'first_appointment']], on='patient_id'), patients[['patient_id', 'name']], on='patient_id')
df_merged['time_gap'] = df_merged['start_date'] - df_merged['first_appointment']
df_merged

,patient_id,start_date,first_appointment,name,time_gap
0,1,2023-05-10,2023-05-10,Ana Torres,0 days
1,2,2023-05-15,2023-05-15,Luis Martínez,0 days
2,5,2023-07-05,2023-07-05,Julia Herrera,0 days
3,4,2023-06-22,2023-06-22,Marta Ruiz,0 days
4,6,2023-09-01,2023-09-01,Pedro López,0 days
5,7,2023-09-10,2023-09-10,Sandra Díaz,0 days
6,8,2023-09-12,2023-09-12,Andrés Pérez,0 days
7,9,2023-10-01,2023-10-01,Laura Castro,0 days
8,10,2023-10-15,2023-10-15,José Molina,0 days
9,11,2023-10-20,2023-10-20,Lucía Peña,0 days


b. What’s the average gap in days for patients who recovered?

In [268]:
#SQL

query = """
WITH first_appointment_patient AS (
SELECT patient_id, MIN(CAST(appointment_date AS date)) AS first_appointment
FROM appointments
GROUP BY patient_id
),

timegap_recovered_patients AS (
SELECT p.name, t.patient_id, t.start_date, fa.first_appointment, (CAST(t.start_date AS date) - fa.first_appointment) AS timegap
FROM treatments t
JOIN first_appointment_patient fa ON fa.patient_id = t.patient_id
JOIN patients p ON p.patient_id = t.patient_id
WHERE t.outcome = 'Recovered'
)

SELECT AVG(timegap)
FROM timegap_recovered_patients
"""

tf.excecute_query(query, engine)

,avg
0,0.0


In [271]:
#Python

treatments_by_patient = treatments.copy()
treatments_by_patient = treatments_by_patient[treatments_by_patient['outcome'] == 'Recovered']
treatments_by_patient['start_date'] = pd.to_datetime(treatments_by_patient['start_date'], format='%Y-%m-%d', errors='coerce')
first_appointment_patient = appointments.copy()
first_appointment_patient['appointment_date'] = pd.to_datetime(first_appointment_patient['appointment_date'], format='%Y-%m-%d', errors='coerce')
first_appointment_patient = first_appointment_patient.groupby('patient_id')['appointment_date'].min().reset_index()
first_appointment_patient = first_appointment_patient.rename(columns={'appointment_date':'first_appointment'})
df_merged = pd.merge(pd.merge(treatments_by_patient[['patient_id', 'start_date']], first_appointment_patient[['patient_id', 'first_appointment']], on='patient_id'), patients[['patient_id', 'name']], on='patient_id')
df_merged['time_gap'] = df_merged['start_date'] - df_merged['first_appointment']
df_merged['time_gap'].mean()

Timedelta('0 days 00:00:00')